# Search for missing RBs in non trace suppressed data with the track trigger

* e.g. Run 118 (without trace suppression) is suitable for such a study
* Run 120 has central track trigger and trace suppression
* Also plot the baselines

In [ ]:
import gaps_online.tof as gt
import gaps_online as go
import gaps_tof as gt_cxx
import gaps_online.db as db
import rust_dataclasses as rd
import re
from tqdm.notebook import tqdm
from glob import glob
from pathlib import Path
#import HErmes.fitting as fit

import dashi as d
import pylab as p
import numpy as np
d.visual()
import charmingbeauty.visual as vis
import charmingbeauty.layout as lo
vis.set_style_present()

# FIXME add DJANGO_ALLOW_ASYNC_UNSAFE=1 to setup.env
RUNPATH  = Path('/data1/nextcloud/cra_data/data/2023_nevis/tof/')
CALIPATH = RUNPATH / 'calibration'

In [ ]:
def rms(data):
    return np.sqrt((1/len(data))*((data**2).sum()))

In [ ]:
paddles = db.tof_paddle_manifest()
rbs     = db.tdb.models.RB.objects.all()
pends   = db.tdb.models.PaddleEnd.objects.all()

In [ ]:
pattern = re.compile('RB(?P<rbid>[0-9]*)_')

calibrations = CALIPATH.glob('*.tof.gaps')
calib = dict()

for fname in calibrations:
    fname = str(fname)
    rbid = int(pattern.search(fname).groupdict()['rbid'])
    calib[rbid] = go.cxx_api.RBCalibration.from_califile(fname)

In [ ]:
print (calib[1])

## Run 120 - central track trigger, trace suppression

In [ ]:
runfiles = (RUNPATH / '120').glob('*.tof.gaps')
nfiles = 0
events = []
for rfile in runfiles:
    packets = gt.get_tofpackets(str(rfile), filter=go.cxx_api.PacketType.TofEvent)
    events.extend([go.cxx_api.TofEvent.from_tofpacket(k) for k in packets])
    nfiles += 1
    if nfiles == 20:
        break

In [ ]:
all_missing_rbs = dict()

less_rbs = 0
more_rbs = 0
for ev in tqdm(events, desc='looking for HG/LG discrepancy...'):
    npaddle = ev.mt_event.n_paddles
    if len(ev.rbevents) != npaddle:
        try:
            hits = ev.mt_event.get_dsi_j_ch()
        except:
            continue
        
        for hit in hits:
            rbhit = db.get_HG_for_LG(*hit)[0]
            if not (rbhit in ev.get_rbids()):
                if not (rbhit in all_missing_rbs):
                    all_missing_rbs[rbhit] = 0
                all_missing_rbs[rbhit] += 1
        if npaddle > len(ev.rbevents):
            less_rbs += 1
        else:
            more_rbs += 1
print(f'\n -- less RB {less_rbs/len(events):1.2f} more RB {more_rbs/len(events):1.2f}, events {len(events)}')
print (all_missing_rbs)

In [ ]:
print (len([k for k in all_missing_rbs]))

In [ ]:
db.get_HG_for_LG(*ev.mt_event.get_dsi_j_ch()[0])
#ev.mt_event.get_dsi_j_ch()

# Summary

We saw that the missing RBs are equally distributed across board ids, both for runs with and without trace suppression.

## Run 118 - no trace suppression

In [ ]:
runfiles = (RUNPATH / '118').glob('*.tof.gaps')
nfiles = 0
events = []
for rfile in runfiles:
    packets = gt.get_tofpackets(str(rfile), filter=go.cxx_api.PacketType.TofEvent)
    events.extend([go.cxx_api.TofEvent.from_tofpacket(k) for k in packets])
    nfiles += 1
    if nfiles == 10:
        break

In [ ]:
ev = events[0]
all_rbs = dict()
all_ev = len(events)
ev_with_missing = 0
for rbid in ev.get_rbids(): 
    all_rbs[rbid] = 0
for ev in events:
    for rbid in ev.get_rbids():
        all_rbs[rbid] += 1
    if len(ev.get_rbids()) != 40:
        ev_with_missing += 1
print (f'-- in total {all_ev} events..')
print (f'-- {ev_with_missing/all_ev:1.4f} with at least 1 RB missing')
print ('-- RBID --> NEvents(RB) -- -- --')
for k in sorted(all_rbs):
    print (f'-- {k:02} --> {all_rbs[k]} [{(all_ev - all_rbs[k])/all_ev:1.4f}%]')


In [ ]:
baselines = dict()
for k in rbs:
    baselines[k.rb_id] = dict()
    for ch in range(0,9):
        baselines[k.rb_id][ch] = []
        
for ev in tqdm(events, desc='Calculating baselines...'):
    for rbid in ev.get_rbids():
        if rbid == 0:
            continue
        rbev  = ev.get_rbevent(rbid)
        volts = calib[rbid].voltages(rbev)
        for ch in rbev.header.get_channels():
            bl = volts[ch][10:50].mean()
            baselines[rbid][ch].append(bl)

for k in baselines:
    for ch in baselines[k]:
        baselines[k][ch] = np.array(baselines[k][ch])
#print (baselines)   

In [ ]:
baseline_rms = dict()
all_baseline_rms = []
for k in rbs:
    baseline_rms[k.rb_id] = dict()
    for ch in range(0,8):
        baseline_rms[k.rb_id][ch] = rms(baselines[k.rb_id][ch])
        all_baseline_rms.append(rms(baselines[k.rb_id][ch]))
for k in baseline_rms:
    print (f'{k} - {baseline_rms[k]}')

In [ ]:
%matplotlib inline
rms_bins = np.linspace(0,50,40)
rms_bins = 30
#print (baseline_rms)
fig = p.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)
ax  = fig.gca()
ax.set_ylim(bottom=0, top=150)
ax.set_xlabel('baseline RMS [mV]', loc='right')
ax.set_ylabel('events', loc='top')
ax.set_yscale('linear')
h = d.histfactory.hist1d(all_baseline_rms,rms_bins)
h.line(alpha=0.8,filled=True)
p.show()

In [ ]:
%matplotlib inline
bl_bins = np.linspace(-3,3,30)
figsize = lo.FIGSIZE_A4[0]*10, lo.FIGSIZE_A4[1]*10
figsize = lo.FIGSIZE_A4
#fig = p.figure(figsize=lo.FIGSIZE_A4_LANDSCAPE)

for rbid in tqdm(baselines, desc='Creating distributions...'):
    fig, axes = p.subplots(nrows=8,
                           ncols=1,
                           sharex=True,
                           sharey=False,
                           figsize=figsize)
    axes = axes.flatten(order='F')
    idx = 0
    for ch in baselines[rbid]:
        if ch == 8:
            continue
        ax = axes[idx]
        p.sca(ax)
        data = baselines[rbid][ch]
        ax.text(-2.9,100,f'RB/CH {rbid}/{ch+1}\nNEVTS {len(data)}', fontsize=8)
        #ax.set_ylim(bottom=0.1)
        #ax.set_ylim(bottom=min(data))
        ax.set_yscale('symlog')
        ax.spines['top'].set_visible(True)
        ax.spines['right'].set_visible(True)
        h   = d.factory.hist1d(data, bl_bins)
        h.line(filled=True, alpha=0.8)
        ax.text(0.8,0.05, 'baseline [mV]',transform=p.gcf().transFigure)
        ax.text(0.015,0.8, 'nevents',transform=p.gcf().transFigure, rotation=90)
        
        idx += 1
        #pass

    fig.subplots_adjust(hspace=0)
    fig.savefig(f'plots/baselines_rb{rbid}.png')
    #break
#p.show()
  

In [ ]:
      
data    = baselines[1][1]

ax  = fig.gca()
ax.set_yscale('symlog')
ax.set_xlabel('baseline bin[10:50] [mV]', loc='right')
ax.set_ylabel('nevents', loc='top')
ax.text(-2.9,max(data)*0.9,'RB/CH 1-1')
h   = d.factory.hist1d(data, bl_bins)
h.line(filled=True, alpha=0.8)
p.show()